In [3]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken
!pip install langchain-community


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 2.2/2.2 MB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 49.2/49.2 kB 1.3 MB/s eta 0:00:00


In [46]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [62]:
import os
import json
with open("E:/ML_Git/config.json") as f:
    config = json.load(f)
api_key = config['openai_api_key_3']

In [48]:
pdfreader = PdfReader('resources/preventive_services_cpcp.pdf') 

In [49]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [50]:
raw_text

" \nA Division of Health Care Service Corporation, a Mutual Legal Reserve Company,  \nan Independent Licensee of the Blue Cross and Blue Shield Association  \n \nIf a conflict arises between a Clinical Payment and Coding Policy (“CPCP”) and any plan document under which a \nmember is entitled to Covered Services, the plan document will govern. If a conflict arises between a CPCP and \nany provider contract pursuant to which a provider participates in and/or provides Covered Services to eligible \nmember(s) and/or plans, the provider contract will govern. “Plan documents” include, but are not limited to, \nCertificates of Health Care Benefits, b enefit booklets, Summary Plan Descriptions, and other coverage documents.  \nBCBS IL may use reasonable discretion interpreting and applying this policy to services being delivered in a \nparticular case. BCBS IL has full and final discretionary authority for th eir interpretation and application to the \nextent provided under any applicable pla

In [51]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [52]:
texts[1]

'Certificates of Health Care Benefits, b enefit booklets, Summary Plan Descriptions, and other coverage documents.  \nBCBS IL may use reasonable discretion interpreting and applying this policy to services being delivered in a \nparticular case. BCBS IL has full and final discretionary authority for th eir interpretation and application to the \nextent provided under any applicable plan documents.  \nProviders are responsible for submission of accurate documentation of services performed.  Providers are \nexpected to submit claims for services rendered using v alid code combinations from Health Insurance Portability \nand Accountability Act (“HIPAA”) approved code sets. Claims should be coded appropriately according to industry'

In [53]:
len(texts)

116

In [54]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings(api_key=api_key)

In [55]:
document_search = FAISS.from_texts(texts, embeddings)

In [56]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [57]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [58]:
query = "What happens when a conflict arises between a CPCP and any provider contract pursuant ?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' In case of a conflict arising between a CPCP and any provider contract pursuant to which a provider participates in and/or provides Covered Services to eligible member(s) and/or plans, the provider contract will govern. '

In [59]:
query = "What are the diagnosis codes that are included in Diagnosis List 1 ?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Z00.00, Z00.01, Z00.110, Z00.111, Z00.121, Z00.129, Z00.8, Z01.411, Z01.419, Z02.83, Z11.1, Z11.3, Z11.4, Z11.51, Z11.7, Z12.11, Z12.12, Z12.2, Z12.31, Z12.39, Z12.4, Z12.5, Z13.0, Z13.1, Z13.220, Z13.31, Z13.32, Z13.4, Z13.41, Z13.42, Z13.5, Z13.6, Z13.820, Z23, Z30.011, Z30.012, Z30.013, Z30.014, Z30.015, Z30.016, Z30.017, Z30.018.'

In [60]:
query = "What are the diagnosis codes that are payable for 77067 ?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' There are multiple diagnosis codes that are payable for 77067, as it depends on the specific context and purpose of the service. However, some possible diagnosis codes that could be payable for 77067 include Z80.3 (Family history of malignant neoplasm of breast), Z80.41 (Family history of malignant neoplasm of ovary), Z85.3 (Personal history of malignant neoplasm of breast), and Z85.43 (Personal history of malignant neoplasm of ovary). It is important to consult the specific guidelines and recommendations for each procedure code to determine the appropriate diagnosis codes for reimbursement.'

In [61]:
query = "What are the diagnosis codes that are payable for Breast Cancer Screening for Women at Average Risk ?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Z80.3 or Z80.41'